In [24]:
import pandas as pd
import numpy as np

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [61]:
df=pd.read_csv(r"D:\Data Science\Notes\titanic\train.csv")

In [62]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [63]:
x=df.iloc[:,1:]
y=df.iloc[:,0]

In [64]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [65]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
313,3,male,28.0,0,0,7.8958,S
714,2,male,52.0,0,0,13.0000,S
383,1,female,35.0,1,0,52.0000,S
279,3,female,35.0,1,1,20.2500,S
879,1,female,56.0,0,1,83.1583,C


In [66]:
# imputation transformer
trf1 = ColumnTransformer(
                        [('impute_age',SimpleImputer(),[2]),
                        ('impute_embark',SimpleImputer(strategy='most_frequent'),[6])],
                        remainder='passthrough')

In [73]:
# one hot encoding
trf2=ColumnTransformer(
                        [('ohe_sex_embark',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])],
                         remainder='passthrough')

In [79]:
#Scaling
trf3=ColumnTransformer(
                       [('scale',MinMaxScaler(),slice(0,10))])

In [80]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [81]:
# train the model
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [82]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [83]:
# train
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embark',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
    

In [84]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embark',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embark',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000002D87487A820>),
 'trf5': DecisionTreeClassifier()}

In [85]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [87]:
# Predict
y_pred = pipe.predict(x_test)

In [88]:
y_pred

array([0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [89]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5977653631284916

# Cross Validation using Pipeline

In [91]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv=5, scoring='accuracy').mean()

0.6460652024032305

# GridSearch using Pipeline

In [92]:
# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [94]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embark',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embark',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000002D87487A820>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [95]:
grid.best_score_

0.6460652024032305

In [96]:

grid.best_params_

{'trf5__max_depth': 1}

# Exporting the Pipeline

In [97]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

# predict-using-pipeline

In [98]:
import pickle
import numpy as np

In [99]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [100]:
# Assume user input
test_input2 = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [101]:
pipe.predict(test_input2)

C:\Users\yoges\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\yoges\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0], dtype=int64)